In [47]:
import psycopg2
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import assets.helpers as hp
import assets.metrics as m
import importlib
from sklearn.model_selection import train_test_split


In [48]:
RANDOM_STATE = 42
# WINDOW_LENGTH = 16
WINDOW_LENGTH = 24
MIN_LOS_ICU = 24
CLIENT_COUNT = 4
USE_FL = False
VITAL_NAMES = ['heartrate', 'sysbp', 'diasbp', 'meanbp', 'resprate', 'tempc', 'spo2']
LAB_NAMES = ['albumin', 'bun', 'bilirubin', 'lactate', 'bicarbonate', 'bands', 'chloride', 'creatinine', 'glucose', 'hemoglobin', 'hematocrit', 'platelet', 'potassium', 'ptt', 'sodium', 'wbc']
FOLDER_SUFFIX = ''
MAX_TIMESTEPS = 10  # Set an upper limit for the number of timesteps

In [49]:
# Connect to db
conn = psycopg2.connect(host='localhost', port=5432, dbname='mimic', user='zainab', password='password')
cur = conn.cursor()

# Read vital signs
vitals = pd.read_sql_query(f'SELECT * FROM mimiciii.vitals_windowed_{WINDOW_LENGTH:d}h;', conn)

# Read in labs values
labs = pd.read_sql_query(f'SELECT * FROM mimiciii.labs_windowed_{WINDOW_LENGTH:d}h;', conn)
# Close the cursor and connection to so the server can allocate bandwidth to other requests
cur.close()
conn.close()

/tmp/ipykernel_44524/3893027086.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vitals = pd.read_sql_query(f'SELECT * FROM mimiciii.vitals_windowed_{WINDOW_LENGTH:d}h;', conn)
/tmp/ipykernel_44524/3893027086.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  labs = pd.read_sql_query(f'SELECT * FROM mimiciii.labs_windowed_{WINDOW_LENGTH:d}h;', conn)


In [50]:
vitals_spec = tf.TensorSpec(
    shape=(None, len(VITAL_NAMES)),
    dtype=tf.dtypes.float64,
    name='vitals'
)
labs_spec = tf.TensorSpec(
    shape=(None, len(LAB_NAMES)),
    dtype=tf.dtypes.float64,
    name='labs'
)
label_spec = tf.TensorSpec(
    shape=1,
    dtype=tf.dtypes.float64,
    name='label'
)

In [51]:
# Convert datetime columns to seconds
for col in vitals.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    vitals[col] = vitals[col].astype(int) / 10**9  # Convert to seconds

for col in labs.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    labs[col] = labs[col].astype(int) / 10**9  # Convert to seconds

# Convert categorical columns to numeric using one-hot encoding
vitals = pd.get_dummies(vitals, drop_first=True)
labs = pd.get_dummies(labs, drop_first=True)

# Merge the vitals and labs data on the common key (icustay_id)
merged_data = pd.merge(vitals, labs, on='icustay_id', suffixes=('_vitals', '_labs'), how='inner')

# Ensure the labels are the same in both datasets
assert all(merged_data['label_death_icu_vitals'] == merged_data['label_death_icu_labs']), "Mismatch in labels between vitals and labs"

# Drop one of the label columns
merged_data = merged_data.drop(columns=['label_death_icu_labs'])

# Rename the remaining label column for consistency
merged_data = merged_data.rename(columns={'label_death_icu_vitals': 'label_death_icu'})


In [52]:
vitals_features = merged_data.filter(like='_vitals')
labs_features = merged_data.filter(like='_labs')
labels = merged_data['label_death_icu']

# Convert to NumPy arrays
X_vitals = vitals_features.to_numpy()
X_labs = labs_features.to_numpy()
y = labels.to_numpy()

# Calculate the number of timesteps that divides the total features without a remainder
def find_timesteps(total_features, max_timesteps):
    for timesteps in range(max_timesteps, 0, -1):
        if total_features % timesteps == 0:
            return timesteps
    return 1  # Fallback to 1 if no valid timesteps found

# Find suitable timesteps for vitals and labs
timesteps_vitals = find_timesteps(X_vitals.shape[1], MAX_TIMESTEPS)
timesteps_labs = find_timesteps(X_labs.shape[1], MAX_TIMESTEPS)

# Calculate number of features per timestep
num_features_vitals = X_vitals.shape[1] // timesteps_vitals
num_features_labs = X_labs.shape[1] // timesteps_labs

# Reshape data into 3D arrays (samples, timesteps, features)
X_vitals = X_vitals.reshape((-1, timesteps_vitals, num_features_vitals))
X_labs = X_labs.reshape((-1, timesteps_labs, num_features_labs))

# Split into training and test sets
X_vitals_train, X_vitals_test, X_labs_train, X_labs_test, y_train, y_test = train_test_split(
    X_vitals, X_labs, y, test_size=0.2, random_state=42)

In [53]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [54]:
# Vital channel
inputs_vitals = tf.keras.Input(shape=vitals_spec.shape, name='Input_vitals')
mask_vitals = tf.keras.layers.Masking(mask_value=-2., name='mask_vitals')(inputs_vitals)
GRU_layer1_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_vitals')(mask_vitals)
GRU_layer2_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_vitals')(GRU_layer1_vitals)
GRU_layer3_vitals = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_vitals')(GRU_layer2_vitals)

# Labs channel
inputs_labs = tf.keras.Input(shape=labs_spec.shape, name='Input_labs')
mask_labs = tf.keras.layers.Masking(mask_value=-2., name='mask_labs')(inputs_labs)
GRU_layer1_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_labs')(mask_labs)
GRU_layer2_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_labs')(GRU_layer1_labs)
GRU_layer3_labs = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_labs')(GRU_layer2_labs)

# Concatenation of both branches
merge = tf.keras.layers.Concatenate()([GRU_layer3_vitals, GRU_layer3_labs])

outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(merge)

model = tf.keras.Model(inputs=[inputs_vitals, inputs_labs], outputs=outputs, name='RNN_model')
model.summary()


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])


Model: "RNN_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_vitals        │ (None, None, 7)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_labs          │ (None, None, 16)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_14        │ (None, None, 7)   │          0 │ Input_vitals[0][… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_15        │ (None, None, 16)  │          0 │ Input_labs[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_vitals         │ (None, None, 7)   │          0 │ Input_vitals[0][… │
│ (Masking)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_14 (Any)        │ (None, None)      │          0 │ not_equal_14[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_labs (Masking) │ (None, None, 16)  │          0 │ Input_labs[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_15 (Any)        │ (None, None)      │          0 │ not_equal_15[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer1_vitals   │ (None, None, 16)  │      1,200 │ mask_vitals[0][0… │
│ (GRU)               │                   │            │ any_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer1_labs     │ (None, None, 16)  │      1,632 │ mask_labs[0][0],  │
│ (GRU)               │                   │            │ any_15[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer2_vitals   │ (None, None, 16)  │      1,632 │ GRU_layer1_vital… │
│ (GRU)               │                   │            │ any_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer2_labs     │ (None, None, 16)  │      1,632 │ GRU_layer1_labs[… │
│ (GRU)               │                   │            │ any_15[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer3_vitals   │ (None, 16)        │      1,632 │ GRU_layer2_vital… │
│ (GRU)               │                   │            │ any_14[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer3_labs     │ (None, 16)        │      1,632 │ GRU_layer2_labs[… │
│ (GRU)               │                   │            │ any_15[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, 32)        │          0 │ GRU_layer3_vital… │
│ (Concatenate)       │                   │            │ GRU_layer3_labs[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ concatenate_7[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,393 (36.69 KB)

 Trainable params: 9,393 (36.69 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# Train the model
history = model.fit(
    x=[X_vitals_train, X_labs_train],
    y=y_train,
    epochs=20,
    batch_size=32,
    validation_data=([X_vitals_test, X_labs_test], y_test)
)

Epoch 1/20


ValueError: Exception encountered when calling GRUCell.call().

[1mDimensions must be equal, but are 1 and 16 for '{{node RNN_model_1/GRU_layer1_labs_1/gru_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](RNN_model_1/GRU_layer1_labs_1/strided_slice_1, RNN_model_1/GRU_layer1_labs_1/gru_cell_1/Cast/ReadVariableOp)' with input shapes: [32,1], [16,48].[0m

Arguments received by GRUCell.call():
  • inputs=tf.Tensor(shape=(32, 1), dtype=float32)
  • states=('tf.Tensor(shape=(32, 16), dtype=float32)',)
  • training=True

In [58]:
# Evaluate the model
results = model.evaluate(
    x=[X_vitals_test, X_labs_test],
    y=y_test
)
print("Test Loss, Test Accuracy, Test Precision, Test Recall, Test F1-Score:", results)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8705 - f1_score: 0.2891 - loss: 0.2865 - precision_4: 0.4877 - recall_4: 0.2164
Test Loss, Test Accuracy, Test Precision, Test Recall, Test F1-Score: [0.3102279007434845, 0.8613037467002869, 0.48571428656578064, 0.17171716690063477, 0.25373128056526184]


In [ ]:


# Constants
WINDOW_LENGTH = 24  # Adjust as needed
MIN_LOS_ICU = 48    # Adjust as needed
MAX_TIMESTEPS = 10  # Set an upper limit for the number of timesteps
CLIENT_COUNT = 4    # Number of clients

# Connect to db
conn = psycopg2.connect(host='localhost', port=5432, dbname='mimic', user='zainab', password='password')
cur = conn.cursor()

# Read vital signs
vitals_query = f'SELECT * FROM mimiciii.vitals_windowed_{WINDOW_LENGTH}h;'
vitals = pd.read_sql_query(vitals_query, conn)

# Read in labs values
labs_query = f'SELECT * FROM mimiciii.labs_windowed_{WINDOW_LENGTH}h;'
labs = pd.read_sql_query(labs_query, conn)

# Close the cursor and connection
cur.close()
conn.close()

# Convert datetime columns to seconds
for col in vitals.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    vitals[col] = vitals[col].astype(int) / 10**9  # Convert to seconds

for col in labs.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    labs[col] = labs[col].astype(int) / 10**9  # Convert to seconds

# Convert categorical columns to numeric using one-hot encoding
vitals = pd.get_dummies(vitals, drop_first=True)
labs = pd.get_dummies(labs, drop_first=True)

# Merge the vitals and labs data on the common key (icustay_id)
merged_data = pd.merge(vitals, labs, on='icustay_id', suffixes=('_vitals', '_labs'), how='inner')

# Ensure the labels are the same in both datasets
assert all(merged_data['label_death_icu_vitals'] == merged_data['label_death_icu_labs']), "Mismatch in labels between vitals and labs"

# Drop one of the label columns
merged_data = merged_data.drop(columns=['label_death_icu_labs'])

# Rename the remaining label column for consistency
merged_data = merged_data.rename(columns={'label_death_icu_vitals': 'label_death_icu'})

vitals_features = merged_data.filter(like='_vitals')
labs_features = merged_data.filter(like='_labs')
labels = merged_data['label_death_icu']

# Convert to NumPy arrays
X_vitals = vitals_features.to_numpy()
X_labs = labs_features.to_numpy()
y = labels.to_numpy()

# Calculate the number of timesteps that divides the total features without a remainder
def find_timesteps(total_features, max_timesteps):
    for timesteps in range(max_timesteps, 0, -1):
        if total_features % timesteps == 0:
            return timesteps
    return 1  # Fallback to 1 if no valid timesteps found

# Find suitable timesteps for vitals and labs
timesteps_vitals = find_timesteps(X_vitals.shape[1], MAX_TIMESTEPS)
timesteps_labs = find_timesteps(X_labs.shape[1], MAX_TIMESTEPS)

# Calculate number of features per timestep
num_features_vitals = X_vitals.shape[1] // timesteps_vitals
num_features_labs = X_labs.shape[1] // timesteps_labs

# Reshape data into 3D arrays (samples, timesteps, features)
X_vitals = X_vitals.reshape((-1, timesteps_vitals, num_features_vitals))
X_labs = X_labs.reshape((-1, timesteps_labs, num_features_labs))

# Split data among clients
def split_data(X_vitals, X_labs, y, num_clients):
    vitals_splits = np.array_split(X_vitals, num_clients)
    labs_splits = np.array_split(X_labs, num_clients)
    labels_splits = np.array_split(y, num_clients)
    return vitals_splits, labs_splits, labels_splits

X_vitals_splits, X_labs_splits, y_splits = split_data(X_vitals, X_labs, y, CLIENT_COUNT)

# Define input shapes
vitals_spec = tf.TensorSpec(
    shape=(timesteps_vitals, num_features_vitals),
    dtype=tf.dtypes.float64,
    name='vitals'
)
labs_spec = tf.TensorSpec(
    shape=(timesteps_labs, num_features_labs),
    dtype=tf.dtypes.float64,
    name='labs'
)

# Define the custom F1-score metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Define the model architecture
def create_model():
    # Vital channel
    inputs_vitals = tf.keras.Input(shape=vitals_spec.shape, name='Input_vitals')
    mask_vitals = tf.keras.layers.Masking(mask_value=-2., name='mask_vitals')(inputs_vitals)
    GRU_layer1_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_vitals')(mask_vitals)
    GRU_layer2_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_vitals')(GRU_layer1_vitals)
    GRU_layer3_vitals = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_vitals')(GRU_layer2_vitals)
    normalized_vitals = tf.keras.layers.BatchNormalization(name='BatchNorm_vitals')(GRU_layer3_vitals)

    # Labs channel
    inputs_labs = tf.keras.Input(shape=labs_spec.shape, name='Input_labs')
    mask_labs = tf.keras.layers.Masking(mask_value=-2., name='mask_labs')(inputs_labs)
    GRU_layer1_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_labs')(mask_labs)
    GRU_layer2_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_labs')(GRU_layer1_labs)
    GRU_layer3_labs = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_labs')(GRU_layer2_labs)
    normalized_labs = tf.keras.layers.BatchNormalization(name='BatchNorm_labs')(GRU_layer3_labs)

    # Concatenation of both branches
    merge = tf.keras.layers.Concatenate()([normalized_vitals, normalized_labs])

    FCL1 = tf.keras.layers.Dense(16, name='FCL1')(merge)
    FCL2 = tf.keras.layers.Dense(16, name='FCL2')(FCL1)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(FCL2)

    model = tf.keras.Model(inputs=[inputs_vitals, inputs_labs], outputs=outputs, name='RNN_model')
    return model

# Train model on each client's data and aggregate the weights
def federated_training(X_vitals_splits, X_labs_splits, y_splits, num_clients, global_epochs, local_epochs, batch_size):
    # Initialize the global model
    global_model = create_model()
    global_model.compile(optimizer='adam',
                         loss='binary_crossentropy',
                         metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])

    for global_epoch in range(global_epochs):
        print(f"Global Epoch {global_epoch+1}/{global_epochs}")

        # Initialize list to store client models
        client_models = []

        for client in range(num_clients):
            print(f" Training on client {client+1}/{num_clients}")
            client_model = create_model()
            client_model.compile(optimizer='adam',
                                 loss='binary_crossentropy',
                                 metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])
            
            # Set the client model weights to the global model weights
            client_model.set_weights(global_model.get_weights())

            # Train the client model
            client_model.fit(x=[X_vitals_splits[client], X_labs_splits[client]],
                             y=y_splits)


: 

In [56]:
import psycopg2
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42
WINDOW_LENGTH = 24
MAX_TIMESTEPS = 10
VITAL_NAMES = ['heartrate', 'sysbp', 'diasbp', 'meanbp', 'resprate', 'tempc', 'spo2']
LAB_NAMES = ['albumin', 'bun', 'bilirubin', 'lactate', 'bicarbonate', 'bands', 'chloride', 'creatinine', 'glucose', 'hemoglobin', 'hematocrit', 'platelet', 'potassium', 'ptt', 'sodium', 'wbc']

# Connect to db
conn = psycopg2.connect(host='localhost', port=5432, dbname='mimic', user='zainab', password='password')
cur = conn.cursor()

# Read vital signs
vitals = pd.read_sql_query(f'SELECT * FROM mimiciii.vitals_windowed_{WINDOW_LENGTH:d}h;', conn)

# Read in labs values
labs = pd.read_sql_query(f'SELECT * FROM mimiciii.labs_windowed_{WINDOW_LENGTH:d}h;', conn)

# Close the cursor and connection to so the server can allocate bandwidth to other requests
cur.close()
conn.close()

# Convert datetime columns to seconds
for col in vitals.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    vitals[col] = vitals[col].astype(int) / 10**9  # Convert to seconds

for col in labs.select_dtypes(include=['datetime64', 'timedelta64']).columns:
    labs[col] = labs[col].astype(int) / 10**9  # Convert to seconds

# Convert categorical columns to numeric using one-hot encoding
vitals = pd.get_dummies(vitals, drop_first=True)
labs = pd.get_dummies(labs, drop_first=True)

# Merge the vitals and labs data on the common key (icustay_id)
merged_data = pd.merge(vitals, labs, on='icustay_id', suffixes=('_vitals', '_labs'), how='inner')

# Ensure the labels are the same in both datasets
assert all(merged_data['label_death_icu_vitals'] == merged_data['label_death_icu_labs']), "Mismatch in labels between vitals and labs"

# Drop one of the label columns
merged_data = merged_data.drop(columns=['label_death_icu_labs'])

# Rename the remaining label column for consistency
merged_data = merged_data.rename(columns={'label_death_icu_vitals': 'label_death_icu'})

vitals_features = merged_data.filter(like='_vitals')
labs_features = merged_data.filter(like='_labs')
labels = merged_data['label_death_icu']

# Convert to NumPy arrays
X_vitals = vitals_features.to_numpy()
X_labs = labs_features.to_numpy()
y = labels.to_numpy()

# Calculate the number of timesteps that divides the total features without a remainder
def find_timesteps(total_features, max_timesteps):
    for timesteps in range(max_timesteps, 0, -1):
        if total_features % timesteps == 0:
            return timesteps
    return 1  # Fallback to 1 if no valid timesteps found

# Find suitable timesteps for vitals and labs
timesteps_vitals = find_timesteps(X_vitals.shape[1], MAX_TIMESTEPS)
timesteps_labs = find_timesteps(X_labs.shape[1], MAX_TIMESTEPS)

# Calculate number of features per timestep
num_features_vitals = X_vitals.shape[1] // timesteps_vitals
num_features_labs = X_labs.shape[1] // timesteps_labs

# Reshape data into 3D arrays (samples, timesteps, features)
X_vitals = X_vitals.reshape((-1, timesteps_vitals, num_features_vitals))
X_labs = X_labs.reshape((-1, timesteps_labs, num_features_labs))

# Split into training and test sets
X_vitals_train, X_vitals_test, X_labs_train, X_labs_test, y_train, y_test = train_test_split(
    X_vitals, X_labs, y, test_size=0.2, random_state=42)

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Vital channel
inputs_vitals = tf.keras.Input(shape=(X_vitals.shape[1], X_vitals.shape[2]), name='Input_vitals')
mask_vitals = tf.keras.layers.Masking(mask_value=-2., name='mask_vitals')(inputs_vitals)
GRU_layer1_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_vitals')(mask_vitals)
GRU_layer2_vitals = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_vitals')(GRU_layer1_vitals)
GRU_layer3_vitals = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_vitals')(GRU_layer2_vitals)

# Labs channel
inputs_labs = tf.keras.Input(shape=(X_labs.shape[1], X_labs.shape[2]), name='Input_labs')
mask_labs = tf.keras.layers.Masking(mask_value=-2., name='mask_labs')(inputs_labs)
GRU_layer1_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer1_labs')(mask_labs)
GRU_layer2_labs = tf.keras.layers.GRU(16, return_sequences=True, name='GRU_layer2_labs')(GRU_layer1_labs)
GRU_layer3_labs = tf.keras.layers.GRU(16, return_sequences=False, name='GRU_layer3_labs')(GRU_layer2_labs)

# Concatenation of both branches
merge = tf.keras.layers.Concatenate()([GRU_layer3_vitals, GRU_layer3_labs])

outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(merge)

model = tf.keras.Model(inputs=[inputs_vitals, inputs_labs], outputs=outputs, name='RNN_model')
model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


/tmp/ipykernel_44524/186642804.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vitals = pd.read_sql_query(f'SELECT * FROM mimiciii.vitals_windowed_{WINDOW_LENGTH:d}h;', conn)
/tmp/ipykernel_44524/186642804.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  labs = pd.read_sql_query(f'SELECT * FROM mimiciii.labs_windowed_{WINDOW_LENGTH:d}h;', conn)


Model: "RNN_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_vitals        │ (None, 4, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_labs          │ (None, 4, 1)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_16        │ (None, 4, 1)      │          0 │ Input_vitals[0][… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_17        │ (None, 4, 1)      │          0 │ Input_labs[0][0]  │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_vitals         │ (None, 4, 1)      │          0 │ Input_vitals[0][… │
│ (Masking)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_16 (Any)        │ (None, 4)         │          0 │ not_equal_16[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_labs (Masking) │ (None, 4, 1)      │          0 │ Input_labs[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_17 (Any)        │ (None, 4)         │          0 │ not_equal_17[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer1_vitals   │ (None, 4, 16)     │        912 │ mask_vitals[0][0… │
│ (GRU)               │                   │            │ any_16[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer1_labs     │ (None, 4, 16)     │        912 │ mask_labs[0][0],  │
│ (GRU)               │                   │            │ any_17[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer2_vitals   │ (None, 4, 16)     │      1,632 │ GRU_layer1_vital… │
│ (GRU)               │                   │            │ any_16[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer2_labs     │ (None, 4, 16)     │      1,632 │ GRU_layer1_labs[… │
│ (GRU)               │                   │            │ any_17[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer3_vitals   │ (None, 16)        │      1,632 │ GRU_layer2_vital… │
│ (GRU)               │                   │            │ any_16[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_layer3_labs     │ (None, 16)        │      1,632 │ GRU_layer2_labs[… │
│ (GRU)               │                   │            │ any_17[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 32)        │          0 │ GRU_layer3_vital… │
│ (Concatenate)       │                   │            │ GRU_layer3_labs[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ concatenate_8[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,385 (32.75 KB)

 Trainable params: 8,385 (32.75 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
# Train the model
history = model.fit(
    x=[X_vitals_train, X_labs_train],
    y=y_train,
    epochs=20,
    batch_size=32,
    validation_data=([X_vitals_test, X_labs_test], y_test)
)

Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.8169 - loss: 0.4793 - precision_14: 0.0968 - recall_14: 0.0414 - val_accuracy: 0.8627 - val_loss: 0.3976 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8581 - loss: 0.4062 - precision_14: 0.0000e+00 - recall_14: 0.0000e+00 - val_accuracy: 0.8627 - val_loss: 0.3903 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8591 - loss: 0.3967 - precision_14: 0.0000e+00 - recall_14: 0.0000e+00 - val_accuracy: 0.8627 - val_loss: 0.3811 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8575 - loss: 0.3930 - precision_14: 0.0000e+00 - recall_14: 0.0000e+00 - val_accuracy: 0.8627 - val_loss: 0.3798 - val_precision_14: 0.0000e+00 - val_recall_14: 0.0000e+00
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0

In [27]:
# Evaluate the model
results = model.evaluate(
    x=[X_vitals_test, X_labs_test],
    y=y_test
)
print("Test Loss, Test Accuracy, Test Precision, Test Recall, Test F1-Score:", results)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9214 - loss: 0.2376 - precision_4: 0.7330 - recall_4: 0.6231
Test Loss, Test Accuracy, Test Precision, Test Recall, Test F1-Score: [0.2630225121974945, 0.9140083193778992, 0.7534246444702148, 0.5555555820465088]
